In [1]:
from google_utils import getKeywordCount
from wordCount import wordCount
import operator

from pymongo import MongoClient
con = MongoClient('127.0.0.1', 27017)
db = con.article_connections
wordCountCache = db.wcCache
wordCountCache.ensure_index("word")

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: ensure_index is deprecated. Use create_index instead.


u'word_1'

In [2]:
import math
def getCachedKeywordCount(word):
    cachedWord = wordCountCache.find_one({"word":word})
    if cachedWord:
        return cachedWord["count"]
    count = getKeywordCount(word)
    wordCountCache.insert({"word":word, "count":count})
    return count

def calidf(counts):
    rst={}
    totalArticleCount=getCachedKeywordCount("a");
    total = 0
    for word, count in counts.iteritems():
        print 'checking word: ' + word
        total += count
        if len(word) == 0:
            continue
        wordArtCount=getCachedKeywordCount(word)
        tfidf = math.log(float(totalArticleCount)/(wordArtCount+1)) *count
        rst[word] = tfidf

    for word, count in rst.iteritems():
        rst[word] = float(count)/total
    return rst

In [3]:
articles = db.englishpod
print articles.count()
articles.find_one()

65


{u'_id': ObjectId('57090efdd46bf97f59c4a6ae'),
 u'content': u'\nA: Good evening. My name is Fabio, I\u2019ll be your waiter for tonight. May I take your order?\nB: No, I\u2019m still working on it. This menu is not even in English. What\u2019s good here?\nA: For you sir, I would recommend spaghetti and meatballs.\nB: Does it come with coke and fries?\nA:It comes with either soup or salad and a complimentary glass of wine, Sir.\nB: I\u2019ll go with the spaghetti and meatballs, salad and the wine.\nA: Excellent choice, your order will be ready soon.\nB: How soon is soon?\nA: Twenty minutes?\nB: You know what? I\u2019ll just go grab a burger across the street.\n',
 u'keyidfs': [[u'meatballs', 0.12526293854113515],
  [u'ill', 0.1093198718545866],
  [u'spaghetti', 0.10026686892797321],
  [u'salad', 0.08426047251369591],
  [u'soon', 0.0742841193509091],
  [u'sir', 0.07059922252510009],
  [u'wine', 0.06292273606070035],
  [u'waiter', 0.05444196530550802],
  [u'fries', 0.05175139835017631],
 

In [4]:
import re, operator
from wordCount import wordCount
file = open('articles.in', 'r')
p = re.compile(r'(^.*\([A-Z][0-9]+\))', re.M)
articlesIn = re.split(p, file.read())
for i in range(len(articlesIn)):
    title = articlesIn[2*i+1]
    content = articlesIn[2*i+2]
    if articles.find_one({'title':title}):
        print title + ' exists!'
        continue
    print '=== Processing ' + title
    counts = wordCount(content)
    keyidfs = calidf(counts)
    sorted_tuple = sorted(keyidfs.items(), key=operator.itemgetter(1))
    articles.insert_one({'title':title, 'content':content, 'keyidfs':sorted_tuple[0:10]})
    print '=== End processing ' + title

Elementary ‐ Difficult Customer (B0001) exists!
Elementary ‐ Calling In Sick (B0002) exists!
Elementary ‐Daily Life ‐ Hotel Upgrade (C0003) exists!
Elementary ‐The Office ‐ I need an assistant! (C0004) exists!
Elementary ‐Daily Life ‐ Cut In Line (C0005) exists!
Elementary ‐ The Weekend ‐ Road Trip (C0006) exists!
Elementary ‐ The Office ‐ Virus! (C0007) exists!
Elementary ‐ Daily Life ‐ What’s your name again? (C0008) exists!
Elementary ‐ The Weekend ‐ Silence please! (C0009) exists!
Elementary ‐ The Office ‐ Driving Sales (C0010) exists!
Elementary ‐ Daily Life ‐ New Guy in Town (C0011) exists!
Elementary ‐ Daily Life ‐ Cleaning the House (C0012) exists!
Elementary ‐ The Office ‐ Out Of Control Spending (C0013) exists!
Elementary ‐ I’m in Debt (B0014) exists!
Elementary ‐ Daily Life ‐ I’m sorry, I love you (C0015) exists!
Elementary ‐ Turn left here! (B0016) exists!
Elementary ‐ Here Comes the Bride (B0017) exists!
Elementary ‐ Upper‐Intermediate ‐ Protest! (D0018) exists!
Elementary

AttributeError: 'NoneType' object has no attribute 'group'